In [1]:
import pandas as pd
import json
import sklearn.svm as svm
import numpy as np

In [12]:
## Change the name to get your path
user = 'Hans'
file1 = 'AG01-01.csv'
file2 = 'GTH01-01.csv'

In [13]:
def load_Data(user, filename):
   data = json.load(open('dir.json'))
   dir = data[user] + filename
#    data = pd.read_csv(dir, header=2)
   return dir

In [51]:
data = pd.read_csv(load_Data(user, file1), header=0)
data['start.time'] = pd.to_datetime(data['start.time'])
data['Class'] = np.random.randint(2, size=len(data))
## selecting the times when the ground truth was observed
observedData = data[(data['start.time'] >= '2017-10-03 16:44:00') & (data['start.time'] <= '2017-10-03 18:44:00')]
observedData['index'] = range(len(observedData))

In [63]:
gt = pd.read_csv(load_Data(user, file2), header=0)
gt['index'] = gt['time']
# gt.head()

In [108]:
observedData = pd.merge(observedData,gt[['index','coding']])
## shuffle the data (for training and testing)
observedData = observedData.sample(frac = 1)

train = observedData[:5000]
test = observedData[5000:]
## this might be an issue since there is ordinality and that makes things weird.  The later parts of this trial
## were more likely to be sedentary.
trainX = train[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']]
trainY = train['coding']
testX = test[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']]
testY = test['coding']

In [109]:
clf = svm.SVC(cache_size=7000)
clf.fit(trainX, trainY)

SVC(C=1.0, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [110]:
## get accuracy of the svm
clf.score(testX,testY)

0.73221387538665483

In [114]:
## what if we tried a neural net for fun?
from sklearn.neural_network import MLPClassifier
n_net = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2000, 1000), random_state=1)
n_net.fit(trainX,trainY)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2000, 1000), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [115]:
## test accuracy of neural net
np.sum(n_net.predict(testX) == testY) / len(testX)

0.66681396376491386